<a href="https://colab.research.google.com/github/Danicoder/Especializacion_IA_BigData/blob/main/UT%207-2%20LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DANIELA GARCÍA MILLÁN

In [ ]:
!pip install -q  einops accelerate bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [ ]:
import torch # PyTorch
import numpy as np
import getpass
import os
import transformers
import accelerate

import warnings
warnings.filterwarnings('ignore')

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

In [ ]:
device

In [ ]:
torch.random.manual_seed(42)

In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

In [ ]:
#id_model = "microsoft/Phi-3-mini-4k-instruct"  # no funciona

id_model = "microsoft/Phi-4-mini-instruct"


In [ ]:
model1 = AutoModelForCausalLM.from_pretrained(id_model, device_map = "cuda", torch_dtype = "auto",
                                             trust_remote_code = True, attn_implementation = "eager")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

In [ ]:
generacion = pipeline("text-generation", model = model1, tokenizer = tokenizer)

In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # from 0.1 to 0.9
    "do_sample": True,
}

In [ ]:
#prompt = "¿Qué es la inteligencia artificial general?"
prompt = "¿Como se hace una tortilla española con pocas palabras?"
#prompt= "Erase una vez un instituto de educacion secundaria donde "
output = generacion(prompt, **generation_args)

In [ ]:
output

In [ ]:
prompt = "what is the result of 7 x 6 - 42?"
output = generacion(prompt, **generation_args)
print(output[0]['generated_text'])

In [ ]:
prompt = "Who was the first woman in space?"
output = generacion(prompt, **generation_args)
print(output[0]['generated_text'])

In [ ]:
prompt = "Una red cnn es "

In [ ]:
template = """<|system|>
You are a helpful assistant and always respond in spanish<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [ ]:
template

In [ ]:
output = generacion(template, **generation_args)
print(output[0]['generated_text'])

In [ ]:
prompt = "Cual fue la primera mujer en el espacio?" # @param {type: "string"}

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = generacion(template, **generation_args)
print(output[0]['generated_text'])

In [ ]:
prompt = "What is AI? Answer in 1 sentence in Spanish in form of a joke " # @param {type:"string"}
#prompt = "What is AI? Answer in 1 sentence" # @param {type:"string"}
#prompt = "What is AI? Answer in the form of a poem" # @param {type: "string"}

sys_prompt = "You are a helpful virtual assistant."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

print(template)

output = generacion(template, **generation_args)
print(output[0]['generated_text'])

In [ ]:
prompt = "Generate a python code that writes the fibonnaci sequence"

sys_prompt = "You are an experienced programmer. Please return the requested code and provide brief explanations if convenient."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

output = generacion(template, **generation_args)
print(output[0]['generated_text'])

In [ ]:
def fibonacci(n):
    # Initialize the first two Fibonacci numbers
    fib_sequence = [0, 1]

    # Generate the Fibonacci sequence up to n terms
    for i in range(2, n):
        next_fib = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_fib)

    return fib_sequence

# Specify the number of terms in the Fibonacci sequence
num_terms = 10

# Generate and print the Fibonacci sequence
fib_sequence = fibonacci(num_terms)
print(fib_sequence)

In [ ]:
def fibonacci(n):
    """
    Generate the Fibonacci sequence up to n terms.

    Parameters:
    n (int): The number of terms to generate.

    Returns:
    list: A list containing the Fibonacci sequence up to n terms.
    """
    fib_sequence = [0, 1]
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return fib_sequence

    for i in range(2, n):
        next_term = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_term)

    return fib_sequence


# Example usage:
n = 10
print(f"Fibonacci sequence up to {n} terms: {fibonacci(n)}")

In [ ]:
prompt = "¿Que es IA generativa?"

msg = [
    {"role": "system", "content": "You are a helpful virtual assistant in data science."},
    {"role": "user", "content": prompt}
]

output = generacion(msg, **generation_args)
print(output[0]["generated_text"])

In [ ]:
prompt = "List 10 famous cities in Spain"
prompt_sys = "You are a helpful travel assistant."

msg = [
    {"role": "system", "content": prompt_sys},
    {"role": "user", "content": prompt},
]

output = generacion(msg, **generation_args)
print(output[0]['generated_text'])

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
model2 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
prompt = ("Who was the first person in space?")
messages = [{"role": "user", "content": prompt}]

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt") # PyTorch
model_inputs = encodeds.to(device)
generated_ids = model2.generate(model_inputs, max_new_tokens = 1000, do_sample = True,
                               pad_token_id = tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]

In [ ]:
res